ACTIVIDAD #5 - NORMALIZACIÓN DE DATOS1




In [2]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import pycountry

In [5]:
url_covid = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
covid_raw = pd.read_csv(url_covid)

print("Diagnóstico inicial:")
display(covid_raw.head())
print(covid_raw.dtypes)
print(covid_raw.isna().sum())


Diagnóstico inicial:


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
                   ...   
3/5/23              int64
3/6/23              int64
3/7/23              int64
3/8/23              int64
3/9/23              int64
Length: 1147, dtype: object
Province/State    198
Country/Region      0
Lat                 2
Long                2
1/22/20             0
                 ... 
3/5/23              0
3/6/23              0
3/7/23              0
3/8/23              0
3/9/23              0
Length: 1147, dtype: int64


# 2. Corrección de columnas


In [6]:
covid_raw.rename(columns={
    "Province/State": "province_state",
    "Country/Region": "country_region",
    "Lat": "lat",
    "Long": "long"
}, inplace=True)


# 3. Pivoteo wide → long


In [16]:
date_columns = covid_raw.columns[4:]
covid_long = covid_raw.melt(
    id_vars=["province_state", "country_region", "lat", "long"],
    value_vars=date_columns,
    var_name="date",
    value_name="confirmed"
)

covid_long["date"] = pd.to_datetime(covid_long["date"], format="%m/%d/%y")



# 4. Normalización de tipos


In [17]:
covid_long["lat"] = covid_long["lat"].astype(float)
covid_long["long"] = covid_long["long"].astype(float)

covid_long["confirmed"] = pd.to_numeric(covid_long["confirmed"], errors="coerce").fillna(0).astype("int64")
covid_long["confirmed"] = covid_long["confirmed"].clip(lower=0)



# 5. Texto y categorías: Enlazar ISO-3


In [10]:
def country_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

covid_long["iso3"] = covid_long["country_region"].apply(country_to_iso3)



# 6. Validaciones


In [18]:
invalid_lat = covid_long[(covid_long["lat"] < -90) | (covid_long["lat"] > 90)]
invalid_long = covid_long[(covid_long["long"] < -180) | (covid_long["long"] > 180)]

print(f"\nFilas con latitud fuera de rango: {len(invalid_lat)}")
print(f"Filas con longitud fuera de rango: {len(invalid_long)}")

display(covid_long.head())




Filas con latitud fuera de rango: 0
Filas con longitud fuera de rango: 0


,province_state,country_region,lat,long,date,confirmed
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0



# PARTE 2: CHIPOTLE
# 1. Ingesta y diagnóstico


In [12]:
url_chipotle = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
chipotle = pd.read_csv(url_chipotle, sep="\t")

print("\nDiagnóstico inicial de Chipotle:")
display(chipotle.head())
print(chipotle.dtypes)
print(chipotle.isna().sum())




Diagnóstico inicial de Chipotle:


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object
order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64


# 2. Normalización de precios y tipos


In [13]:
chipotle["item_price_num"] = chipotle["item_price"].replace("[\$,]", "", regex=True).astype(float)
chipotle.rename(columns={"item_price_num": "unit_price"}, inplace=True)

chipotle["quantity"] = chipotle["quantity"].astype(int).clip(lower=1)

chipotle["line_total"] = round(chipotle["unit_price"] * chipotle["quantity"], 2)


<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-149209875.py:1: SyntaxWarning: invalid escape sequence '\$'
  chipotle["item_price_num"] = chipotle["item_price"].replace("[\$,]", "", regex=True).astype(float)


# 3. Limpieza de texto y categorías


In [14]:
chipotle["item_name_norm"] = chipotle["item_name"].str.lower().str.strip()



# 4. Validaciones


In [15]:
assert (chipotle["unit_price"] >= 0).all(), "Error: hay precios negativos"
assert (chipotle["quantity"] >= 1).all(), "Error: hay cantidades inválidas"

from scipy import stats

chipotle["zscore"] = chipotle.groupby("item_name_norm")["unit_price"].transform(
    lambda x: stats.zscore(x, nan_policy="omit")
)
outliers = chipotle[np.abs(chipotle["zscore"]) > 3]

print(f"\nCantidad de outliers detectados: {len(outliers)}")
display(outliers.head())

display(chipotle.head())


Cantidad de outliers detectados: 123


/tmp/ipython-input-3974398189.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  lambda x: stats.zscore(x, nan_policy="omit")
/tmp/ipython-input-3974398189.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  lambda x: stats.zscore(x, nan_policy="omit")
/tmp/ipython-input-3974398189.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  lambda x: stats.zscore(x, nan_policy="omit")
/tmp/ipython-input-3974398189.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  lambda x: stats.zscore(x, nan

,order_id,quantity,item_name,choice_description,item_price,unit_price,line_total,item_name_norm,zscore
217,96,2,Chicken Crispy Tacos,"[Tomatillo Red Chili Salsa, [Cheese, Sour Crea...",$17.50,17.50,35.00,chicken crispy tacos,3.333745
223,98,2,Chips,NaN,$4.30,4.30,8.60,chips,3.009259
235,103,2,Chips and Tomatillo Green Chili Salsa,NaN,$5.90,5.90,11.80,chips and tomatillo green chili salsa,4.527693
281,123,2,Steak Salad Bowl,"[Tomatillo Red Chili Salsa, [Black Beans, Chee...",$23.78,23.78,47.56,steak salad bowl,3.458469
353,152,2,Steak Burrito,"[Fresh Tomato (Mild), [Lettuce, Guacamole, Ric...",$22.16,22.16,44.32,steak burrito,4.552941


,order_id,quantity,item_name,choice_description,item_price,unit_price,line_total,item_name_norm,zscore
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,2.39,2.39,chips and fresh tomato salsa,-0.220638
1,1,1,Izze,[Clementine],$3.39,3.39,3.39,izze,NaN
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39,3.39,nantucket nectar,-0.282843
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,2.39,2.39,chips and tomatillo-green chili salsa,-0.262613
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98,33.96,chicken bowl,2.759581
